# Feature Engineering and Missing Values

In [34]:
import pandas as pd
from sklearn import impute, preprocessing
import numpy as np

def encode(
    values: np.ndarray, *, encoder: preprocessing.LabelEncoder = None
) -> np.ndarray:
    if encoder is None:
        encoder = preprocessing.LabelEncoder()
        encoder.fit(values)

    return encoder.fit_transform(values)

## Combine Train and Test

In [35]:
# Read in the two datasets
tr_df = pd.read_csv("./data/train.csv", index_col="PassengerId")
te_df = pd.read_csv("./data/test.csv", index_col="PassengerId")

# Take out labels from training data
survived = tr_df["Survived"].copy()
tr_df = tr_df.drop("Survived", axis=1)

# Store indices
tr_index = tr_df.index
te_index = te_df.index

df = pd.concat([tr_df, te_df])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Using Presence of Missing Values As A Feature

A new feature called `missing_<feature>` is added which is True if the row had missing values for that particular feature before imputing them.

In [36]:
df["missing_Age"] = df["Age"].isna()
df["missing_Cabin"] = df["Cabin"].isna()
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,missing_Age,missing_Cabin
PassengerId,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False,True
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False,False
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,True
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False,False
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False,True


## Family Size and Alone

The `SibSp` and `Parch` columns are added together to make a new feature called `FamilySize`. `1` is added to account for the passenger itself. Using this, another feature is added called `Alone`, which is `True` when `FamilySize` is `1`.

In [37]:
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
df["Alone"] = df["FamilySize"] == 1
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,missing_Age,missing_Cabin,FamilySize,Alone
PassengerId,,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False,True,2,False
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False,False,2,False
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,True,1,True
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False,False,2,False
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False,True,1,True


## Group Size

This features captures information about those people who might not be travelling with their family but have maids, manservants, etc., travelling on the same ticket.

In [38]:
df["GroupSize"] = df.groupby("Ticket")["Ticket"].transform("count")
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,missing_Age,missing_Cabin,FamilySize,Alone,GroupSize
PassengerId,,,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False,True,2,False,1
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False,False,2,False,2
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,True,1,True,1
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False,False,2,False,2
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False,True,1,True,1


## Title

The Title (`Mr`, `Mrs` etc.) is extracted from the Name feature.

In [39]:
df['Title'] = df['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,missing_Age,missing_Cabin,FamilySize,Alone,GroupSize,Title
PassengerId,,,,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False,True,2,False,1,Mr
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False,False,2,False,2,Mrs
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,True,1,True,1,Miss
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False,False,2,False,2,Mrs
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False,True,1,True,1,Mr


## Married

The `Married` feature is created which is `True` for any passenger with the title `Mrs`.

In [40]:
df["Married"] = df["Title"] == "Mrs"
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,missing_Age,missing_Cabin,FamilySize,Alone,GroupSize,Title,Married
PassengerId,,,,,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False,True,2,False,1,Mr,False
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False,False,2,False,2,Mrs,True
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,True,1,True,1,Miss,False
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False,False,2,False,2,Mrs,True
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False,True,1,True,1,Mr,False


## Droppinig Unnecessary Columns and Encoding data

The Name, Ticket and Cabin features are dropped and the following features are encoded as categorical:

- Sex
- Embarked
- Pclass
- FamilySize
- Alone
- Title
- Married
- All the `missing_` columns
- 
There are only two missing values in Embarked. These will be filled with the most frequent.

In [41]:
drop = ["Name", "Ticket", "Cabin"]
df = df.drop(drop, axis=1)
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,missing_Age,missing_Cabin,FamilySize,Alone,GroupSize,Title,Married
PassengerId,,,,,,,,,,,,,,
1,3,male,22.0,1,0,7.2500,S,False,True,2,False,1,Mr,False
2,1,female,38.0,1,0,71.2833,C,False,False,2,False,2,Mrs,True
3,3,female,26.0,0,0,7.9250,S,False,True,1,True,1,Miss,False
4,1,female,35.0,1,0,53.1000,S,False,False,2,False,2,Mrs,True
5,3,male,35.0,0,0,8.0500,S,False,True,1,True,1,Mr,False


In [42]:
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode().iloc[0])

to_be_encoded = ["Sex", "Pclass", "Embarked", "Alone", "Title", "Married"]
to_be_encoded.extend(col for col in df.columns if col.startswith("missing_"))

for column in to_be_encoded:
    print(f"Encoding {column}")
    df[column] = encode(df[column].values)
    
df.head()

Encoding Sex
Encoding Pclass
Encoding Embarked
Encoding Alone
Encoding Title
Encoding Married
Encoding missing_Age
Encoding missing_Cabin


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,missing_Age,missing_Cabin,FamilySize,Alone,GroupSize,Title,Married
PassengerId,,,,,,,,,,,,,,
1,2,1,22.0,1,0,7.2500,2,0,1,2,0,1,12,0
2,0,0,38.0,1,0,71.2833,0,0,0,2,0,2,13,1
3,2,0,26.0,0,0,7.9250,2,0,1,1,1,1,9,0
4,0,0,35.0,1,0,53.1000,2,0,0,2,0,2,13,1
5,2,1,35.0,0,0,8.0500,2,0,1,1,1,1,12,0


In [43]:
df.shape

(1309, 14)

## Split and Impute Missing Values

### Training Data

This is done separately to maintain the distribution difference between the two datasets.

In [44]:
# Train data
tr_df = df.loc[tr_index, :]
tr_df["Survived"] = survived

In [45]:
# Scale for KNNImputer
scaler = preprocessing.MinMaxScaler()
cols = ["Pclass", "SibSp", "Parch", "Fare", "Age", "Sex", "FamilySize", "GroupSize"]
temp = tr_df[cols]
temp = scaler.fit_transform(temp)

In [46]:
# Impute
imputer = impute.KNNImputer(n_neighbors=5)
imputed = imputer.fit_transform(temp)
tr_df[cols] = scaler.inverse_transform(imputed)
tr_df.isna().any()

Pclass           False
Sex              False
Age              False
SibSp            False
Parch            False
Fare             False
Embarked         False
missing_Age      False
missing_Cabin    False
FamilySize       False
Alone            False
GroupSize        False
Title            False
Married          False
Survived         False
dtype: bool

In [47]:
tr_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,missing_Age,missing_Cabin,FamilySize,Alone,GroupSize,Title,Married,Survived
PassengerId,,,,,,,,,,,,,,,
1,2.0,1.0,22.0,1.0,0.0,7.2500,2,0,1,2.0,0,1.0,12,0,0
2,0.0,0.0,38.0,1.0,0.0,71.2833,0,0,0,2.0,0,2.0,13,1,1
3,2.0,0.0,26.0,0.0,0.0,7.9250,2,0,1,1.0,1,1.0,9,0,1
4,0.0,0.0,35.0,1.0,0.0,53.1000,2,0,0,2.0,0,2.0,13,1,1
5,2.0,1.0,35.0,0.0,0.0,8.0500,2,0,1,1.0,1,1.0,12,0,0


### Test Data

In [50]:
# Test data
te_df = df.loc[te_index, :]

In [51]:
# Scale for KNNImputer
scaler = preprocessing.MinMaxScaler()
cols = ["Pclass", "SibSp", "Parch", "Fare", "Age", "Sex", "FamilySize", "GroupSize"]
temp = te_df[cols]
temp = scaler.fit_transform(temp)

In [52]:
# Impute
imputer = impute.KNNImputer(n_neighbors=5)
imputed = imputer.fit_transform(temp)
te_df[cols] = scaler.inverse_transform(imputed)
te_df.isna().any()

Pclass           False
Sex              False
Age              False
SibSp            False
Parch            False
Fare             False
Embarked         False
missing_Age      False
missing_Cabin    False
FamilySize       False
Alone            False
GroupSize        False
Title            False
Married          False
dtype: bool

In [53]:
te_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,missing_Age,missing_Cabin,FamilySize,Alone,GroupSize,Title,Married
PassengerId,,,,,,,,,,,,,,
892,2.0,1.0,34.5,0.0,0.0,7.8292,1,0,1,1.0,1,1.0,12,0
893,2.0,0.0,47.0,1.0,0.0,7.0000,2,0,1,2.0,0,1.0,13,1
894,1.0,1.0,62.0,0.0,0.0,9.6875,1,0,1,1.0,1,1.0,12,0
895,2.0,1.0,27.0,0.0,0.0,8.6625,2,0,1,1.0,1,1.0,12,0
896,2.0,0.0,22.0,1.0,1.0,12.2875,2,0,1,3.0,0,2.0,13,1


## Save the new datasets

In [54]:
tr_df = tr_df.reset_index()
tr_df.to_csv("./data/final_train.csv", index=False)

te_df = te_df.reset_index()
te_df.to_csv("./data/final_test.csv", index=False)